In [1]:
import pandas as pd
import sqlite3
import pyodbc

DB = {'servername': r'LAPTOP-NDOACQTH\SQLEXPRESS',
      'database': 'PR4'}

# Constructing the connection string with Windows Authentication
conn_str = (
    'Driver={SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'Trusted_Connection=yes;'
)

# Establishing the connection
export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor

In [2]:
#inventory_levels
inventory_levels_df = pd.read_csv('../GO_SALES_INVENTORY_LEVELSData.csv', index_col=False)
goSales = sqlite3.connect('../go_sales.sqlite')
product_df = pd.read_sql_query("SELECT PRODUCT_NUMBER, MARGIN, LANGUAGE, DESCRIPTION, INTRODUCTION_DATE FROM product", goSales)
inventory_levels_df['PRODUCT_NUMBER'] = inventory_levels_df['PRODUCT_NUMBER'].astype(str)

# Merge inventory_levels_df with product_df on PRODUCT_NUMBER
merged_df = pd.merge(inventory_levels_df, product_df, on='PRODUCT_NUMBER', how='left')

# Replace NaN values in MARGIN, LANGUAGE, and DESCRIPTION with empty strings
merged_df[['MARGIN', 'LANGUAGE', 'DESCRIPTION']] = merged_df[['MARGIN', 'LANGUAGE', 'DESCRIPTION']].fillna('')

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the merged DataFrame and insert them into the database
for index, row in merged_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[INVENTORY_SALES_LEVELS] (INVENTORY_YEAR, INVENTORY_MONTH, PRODUCT_NUMBER, INVENTORY_COUNT, MARGIN, LANGUAGE, DESCRIPTION) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   row['INVENTORY_YEAR'], row['INVENTORY_MONTH'], row['PRODUCT_NUMBER'], row['INVENTORY_COUNT'], row['MARGIN'], row['LANGUAGE'], row['DESCRIPTION'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()


In [3]:
#sales_product_forecast
sales_product_forecast_df = pd.read_csv('../GO_SALES_PRODUCT_FORECASTData.csv', index_col=False)

sales_product_forecast_df['PRODUCT_NUMBER'] = sales_product_forecast_df['PRODUCT_NUMBER'].astype(str)

# Merge inventory_levels_df with product_df on PRODUCT_NUMBER
merged_df = pd.merge(sales_product_forecast_df, product_df, on='PRODUCT_NUMBER', how='left')

# Replace NaN values in MARGIN, LANGUAGE, and DESCRIPTION with empty strings
merged_df[['MARGIN', 'LANGUAGE', 'DESCRIPTION']] = merged_df[['MARGIN', 'LANGUAGE', 'DESCRIPTION']].fillna('')

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the merged DataFrame and insert them into the database
for index, row in merged_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[SALES_PRODUCT_FORECAST] (YEAR, MONTH, PRODUCT_NUMBER, EXPECTED_VOLUME, MARGIN, LANGUAGE, DESCRIPTION) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   row['YEAR'], row['MONTH'], row['PRODUCT_NUMBER'], row['EXPECTED_VOLUME'], row['MARGIN'], row['LANGUAGE'], row['DESCRIPTION'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()

In [4]:
#returned_item_df
goSales = sqlite3.connect('../go_sales.sqlite')

# Read data from the 'returned_item' table into a DataFrame
returned_item_df = pd.read_sql_query("SELECT RETURN_CODE, RETURN_DATE, ORDER_DETAIL_CODE, RETURN_REASON_CODE, RETURN_QUANTITY FROM returned_item", goSales)

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the DataFrame and insert them into the SQL Server database
for index, row in returned_item_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[RETURNED_ITEM] (RETURN_CODE, RETURN_DATE, ORDER_NUMBER, RETURN_REASON_CODE, RETURN_QUANTITY) VALUES (?, ?, ?, ?, ?)",
                   row['RETURN_CODE'], row['RETURN_DATE'], row['ORDER_DETAIL_CODE'], row['RETURN_REASON_CODE'], row['RETURN_QUANTITY'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()

In [6]:
#Sales_target_data
goSales = sqlite3.connect('../go_sales.sqlite')

# Read data from the 'SALES_TARGETData' table into a DataFrame
sales_targetdata_df = pd.read_sql_query("SELECT Id, SALES_STAFF_CODE, SALES_YEAR, SALES_PERIOD, RETAILER_NAME, PRODUCT_NUMBER, SALES_TARGET, RETAILER_CODE FROM SALES_TARGETData", goSales)

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Merge with product_df on PRODUCT_NUMBER to get additional columns
merged_sales_targetdata_df = pd.merge(sales_targetdata_df, product_df, on='PRODUCT_NUMBER', how='left')

# Iterate over the rows of the DataFrame and insert them into the SQL Server database
for index, row in merged_sales_targetdata_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[SALES_TARGET_DATA] (SALES_TARGET_DATA_id, SALES_STAFF_CODE, SALES_YEAR, SALES_PERIODS, RETAILER_NAME, PRODUCT_NUMBER, SALES_TARGET, INTRODUCTION_DATE, RETAILER_CODE, MARGIN, LANGUAGE, DESCRIPTION) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   row['Id'], row['SALES_STAFF_CODE'], row['SALES_YEAR'], row['SALES_PERIOD'], row['RETAILER_NAME'], row['PRODUCT_NUMBER'], row['SALES_TARGET'], row['INTRODUCTION_DATE'], row['RETAILER_CODE'], row['MARGIN'], row['LANGUAGE'], row['DESCRIPTION'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()

In [8]:
#Satisfaction
goStaff = sqlite3.connect('../go_staff.sqlite')

satisfaction_df = pd.read_sql_query("SELECT YEAR, SALES_STAFF_CODE, SATISFACTION_TYPE_CODE FROM SATISFACTION", goStaff)

satisfaction_type_df = pd.read_sql_query("SELECT SATISFACTION_TYPE_CODE, SATISFACTION_TYPE_DESCRIPTION FROM satisfaction_type", goStaff)

sales_staff_df = pd.read_sql_query("SELECT SALES_STAFF_CODE, POSITION_EN, EXTENSION, DATE_HIRED FROM sales_staff", goStaff)

merged_satisfaction_df = pd.merge(satisfaction_df, satisfaction_type_df, on='SATISFACTION_TYPE_CODE', how='left')

final_merged_df = pd.merge(merged_satisfaction_df, sales_staff_df, on='SALES_STAFF_CODE', how='left')

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the DataFrame and insert them into the SQL Server database
for index, row in final_merged_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[SATISFACTION] (YEAR, SALES_STAFF_CODE, SATISFACTION_TYPE_CODE, SATISFACTION_TYPE_DESCRIPTION, POSITION_EN, EXTENSION, DATE_HIRED) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   row['YEAR'], row['SALES_STAFF_CODE'], row['SATISFACTION_TYPE_CODE'], row['SATISFACTION_TYPE_DESCRIPTION'], row['POSITION_EN'], row['EXTENSION'], row['DATE_HIRED'])

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()

IntegrityError: ('23000', "[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]Violation of PRIMARY KEY constraint 'PK_SATISFACTION'. Cannot insert duplicate key in object 'dbo.SATISFACTION'. The duplicate key value is (2004, 100). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)")

In [9]:
#SALES_DEMOGRAPHIC
goCrm = sqlite3.connect('../go_crm.sqlite')

# Read data from the 'SALES_DEMOGRAPHIC' table into a DataFrame
SALES_DEMOGRAPHIC_df = pd.read_sql_query("SELECT DEMOGRAPHIC_CODE, RETAILER_CODEMR, AGE_GROUP_CODE, SALES_PERCENT FROM SALES_DEMOGRAPHIC", goCrm)

# Read data from the 'AGE_GROUP' table into a DataFrame
AGE_GROUP_df = pd.read_sql_query("SELECT UPPER_AGE, LOWER_AGE, AGE_GROUP_CODE FROM AGE_GROUP", goCrm)

# Read data from the 'RETAILER' table into a DataFrame
RETAILER_df = pd.read_sql_query("SELECT COMPANY_NAME, RETAILER_CODEMR FROM RETAILER", goCrm)

# Merge SALES_DEMOGRAPHIC_df with AGE_GROUP_df on AGE_GROUP_CODE to get additional columns
merged_sales_demographic_df = pd.merge(SALES_DEMOGRAPHIC_df, AGE_GROUP_df, on='AGE_GROUP_CODE', how='left')

# Merge merged_sales_demographic_df with RETAILER_df on RETAILER_CODEMR to get additional columns
final_merged_df = pd.merge(merged_sales_demographic_df, RETAILER_df, on='RETAILER_CODEMR', how='left')

# Establish connection to SQL Server database
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()

# Iterate over the rows of the DataFrame and insert them into the SQL Server database
for index, row in final_merged_df.iterrows():
    cursor.execute("INSERT INTO [PR4].[dbo].[SALES_DEMOGRAPHIC] (DEMOGRAPHIC_CODE, UPPER_AGE, LOWER_AGE, AGE_GROUP_CODE, SALES_PERCENT, COMPANY_NAME, RETAILER_CODEMR) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   str(row['DEMOGRAPHIC_CODE']), str(row['UPPER_AGE']), str(row['LOWER_AGE']), str(row['AGE_GROUP_CODE']), str(row['SALES_PERCENT']), str(row['COMPANY_NAME']), str(row['RETAILER_CODEMR']))

# Commit the transaction and close the cursor and connection
export_conn.commit()
cursor.close()
export_conn.close()

In [10]:
go_staff = sqlite3.connect('../go_staff.sqlite')
trainingDF = pd.read_sql_query("SELECT * FROM TRAINING",go_staff)
courseDF = pd.read_sql_query("SELECT * FROM COURSE", go_staff)
staffDF = pd.read_sql_query("SELECT * FROM SALES_STAFF", go_staff)
merged_trainingCourse_df = pd.merge(trainingDF, courseDF, on='COURSE_CODE')
merged_helemaal = pd.merge(merged_trainingCourse_df, staffDF, on='SALES_STAFF_CODE')
export_conn = pyodbc.connect(conn_str)
cursor = export_conn.cursor()
for index, row in merged_helemaal.iterrows():    
    cursor.execute("INSERT INTO [PR4].[dbo].[TRAINING]([YEAR],[SALES_STAFF_CODE], [COURSE_CODE], [COURSE_DESCRIPTION],[POSITION_EN],[EXTENSION],[DATE_HIRED]) VALUES (?, ?, ?,?,?,?,?)", row["YEAR"], row["SALES_STAFF_CODE"],row["COURSE_CODE"],row["COURSE_DESCRIPTION"],row["POSITION_EN"],row["EXTENSION"],row["DATE_HIRED"])    

export_conn.commit()